In [1]:
#import packages
import requests
import json
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/roadevents?api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [3]:
#summarize the data by park
park_list = []
for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            park_list.append(y['organization_name'])

park_df = Counter(park_list)
park_df = pd.DataFrame.from_dict(park_df, orient="index").reset_index().rename(
    columns={'index': 'park',
            0:'number of alerts'}).sort_values(by=['number of alerts'])


In [4]:
# Get today's date as a string
today = datetime.date.today()
today_string = today.strftime('%Y-%m-%d')

park_df['date'] = today_string

In [5]:
len(park_df), sum(park_df['number of alerts']), max(park_df['number of alerts'])

(14, 62, 14)

In [6]:
# Loop through the API entries
counter=0

park_list = []
alert_name_list = []
description_list = []
event_type_list = []
road_name_list = []
vehicle_impact_list = []
start_date_list = []
end_date_list = []
today_date_list = []
id_list = []


for x in df["features"]:
    for y in df["road_event_feed_info"]['data_sources']:
        if y['data_source_id']==x['properties']['core_details']['data_source_id']:
            print("Park:", y['organization_name'])
            park_list.append(y['organization_name'])
    print("Alert name: ", x['properties']['core_details']['name'])
    print("Description: ", x['properties']['core_details']['description'])
    print("Event type: ", x['properties']['core_details']['event_type'])
    print("Road name: ", x['properties']['core_details']['road_names'])
    print("Vehicle impact: ", x['properties']['vehicle_impact'])
    print("Alert ID: ", x['properties']['core_details']['data_source_id'])
  #  print("Dates: ", x['properties']['start_date'], "to ", x['properties']['end_date'])
    print("")
    alert_name_list.append(x['properties']['core_details']['name'])
    description_list.append(x['properties']['core_details']['description'])
    event_type_list.append(x['properties']['core_details']['event_type'])
    road_name_list.append(x['properties']['core_details']['road_names'])
    vehicle_impact_list.append(x['properties']['vehicle_impact'])
    start_date_list.append(x['properties']['start_date'])
    id_list.append(x['properties']['core_details']['data_source_id'])
   # end_date_list.append(x['properties']['end_date'])
    today_date_list.append(today_string)
    counter+=1

Park: Crater Lake National Park
Alert name:  Road from Park Headquarters to Rim Village is Temporarily CLOSED
Description:  The road from Park Headquarters to Rim Village is temporarily closed due to the latest snowstorm.
Event type:  incident
Road name:  ['Munson Valley Road from Park Headquarters to Rim Village']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Road from Park Headquarters to Rim Village is Temporarily CLOSED
Description:  The road from Park Headquarters to Rim Village is temporarily closed due to the latest snowstorm.
Event type:  incident
Road name:  ['Munson Valley Road from Park Headquarters to Rim Village']
Vehicle impact:  all-lanes-closed
Alert ID:  02e0312b-cbd4-40bb-2c4e-08d4c7ad36e2

Park: Crater Lake National Park
Alert name:  Seasonal Road Closure
Description:  North Entrance Road, West Rim Drive, and East Rim Drive are CLOSED for the season.  
Event type:  incident
Road name:  

In [7]:
df_update = pd.DataFrame({
    "date":today_date_list,
    "park":park_list,
    "alert name": alert_name_list,
    "description": description_list,
    "event type": event_type_list,
    "road name": road_name_list,
    "vehicle impact": vehicle_impact_list,
    "event start date": start_date_list,
    "alert_id": id_list
#    "event end date": end_date_list
})
df_update.head

<bound method NDFrame.head of           date                                        park  \
0   2024-04-02                   Crater Lake National Park   
1   2024-04-02                   Crater Lake National Park   
2   2024-04-02                   Crater Lake National Park   
3   2024-04-02                   Crater Lake National Park   
4   2024-04-02                   Crater Lake National Park   
5   2024-04-02                   Crater Lake National Park   
6   2024-04-02                   Crater Lake National Park   
7   2024-04-02                   Crater Lake National Park   
8   2024-04-02                   Crater Lake National Park   
9   2024-04-02                   Crater Lake National Park   
10  2024-04-02                   Crater Lake National Park   
11  2024-04-02                   Crater Lake National Park   
12  2024-04-02                   Crater Lake National Park   
13  2024-04-02                   Crater Lake National Park   
14  2024-04-02                    Shenan

In [8]:
path = "Park Unit Scraping Information.csv"
regionlookup = pd.read_csv(path, encoding='latin-1')
regionlookup.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [9]:
regionlookup.rename(columns={'Park Full Name':"park", "Reg":"region","Alpha":"alpha"}, inplace = True)
region_subset = regionlookup[['park', 'region', 'alpha', "State"]]
region_subset.head

<bound method NDFrame.head of                                                   park region alpha  \
0    Abraham Lincoln Birthplace National Historical...    SER  ABLI   
1                                 Acadia National Park    NER  ACAD   
2                       Adams National Historical Park    NER  ADAM   
3              African Burial Ground National Monument    NER  AFBG   
4                  Agate Fossil Beds National Monument    MWR  AGFO   
5                                   Alagnak Wild River    AKR  ALAG   
6            Alibates Flint Quarries National Monument    IMR  ALFL   
7                  Ala Kahakai National Historic Trail     PW  ALKA   
8    Allegheny Portage Railroad National Historic Site    NER  ALPO   
9                     Amistad National Recreation Area    IMR  AMIS   
10                              American Memorial Park     PW  AMME   
11                Andersonville National Historic Site    SER  ANDE   
12            Aniakchak National Monument and P

In [10]:
merged_df = pd.merge(df_update, region_subset, on="park", how="inner")
merged_df.head

<bound method NDFrame.head of           date                                        park  \
0   2024-04-02                   Crater Lake National Park   
1   2024-04-02                   Crater Lake National Park   
2   2024-04-02                   Crater Lake National Park   
3   2024-04-02                   Crater Lake National Park   
4   2024-04-02                   Crater Lake National Park   
5   2024-04-02                   Crater Lake National Park   
6   2024-04-02                   Crater Lake National Park   
7   2024-04-02                   Crater Lake National Park   
8   2024-04-02                   Crater Lake National Park   
9   2024-04-02                   Crater Lake National Park   
10  2024-04-02                   Crater Lake National Park   
11  2024-04-02                   Crater Lake National Park   
12  2024-04-02                   Crater Lake National Park   
13  2024-04-02                   Crater Lake National Park   
14  2024-04-02                    Shenan

In [11]:
df_old_update = pd.read_csv("./TDx_tracker.csv")

In [12]:
df_update = pd.concat([merged_df, df_old_update])
df_update.head

<bound method NDFrame.head of           date                                        park  \
0   2024-04-02                   Crater Lake National Park   
1   2024-04-02                   Crater Lake National Park   
2   2024-04-02                   Crater Lake National Park   
3   2024-04-02                   Crater Lake National Park   
4   2024-04-02                   Crater Lake National Park   
5   2024-04-02                   Crater Lake National Park   
6   2024-04-02                   Crater Lake National Park   
7   2024-04-02                   Crater Lake National Park   
8   2024-04-02                   Crater Lake National Park   
9   2024-04-02                   Crater Lake National Park   
10  2024-04-02                   Crater Lake National Park   
11  2024-04-02                   Crater Lake National Park   
12  2024-04-02                   Crater Lake National Park   
13  2024-04-02                   Crater Lake National Park   
14  2024-04-02                    Shenan

In [13]:
null_region_count = merged_df['region'].isnull().sum()
null_region_count

0

In [14]:
if null_region_count == 0:
    merged_df.to_csv("./TDx_tracker.csv", index=False)
else:
    null_region_rows = joined[joined['region'].isnull()]
    print("Rows with a null Region column:")
    print(null_region_rows)

In [15]:
path = "Park Unit Scraping Information.csv"
parks = pd.read_csv(path, encoding='latin-1')
parks.head()

,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,Abraham Lincoln Birthplace,Abraham Lincoln Birthplace National Historical...,National Historical Park,KY,SER,ABLI,1.0
1,Acadia,Acadia National Park,National Park,ME,NER,ACAD,1.0
2,Adams,Adams National Historical Park,National Historical Park,MA,NER,ADAM,1.0
3,African Burial Ground,African Burial Ground National Monument,National Monument,NY,NER,AFBG,1.0
4,Agate Fossil Beds,Agate Fossil Beds National Monument,National Monument,NE,MWR,AGFO,1.0


In [16]:
#pull API using API key
response_API = requests.get('https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=3AidKP0PiqgfP8sbwZC0RZ9JM6QDnvYaZx3RRjrH')
#print(response_API.status_code)
data = response_API.text
df = json.loads(data)

In [17]:
id_list = []
url_list = []
title_list = []
parkCode_list = []
description_list = []
category_list = []
relatedRoadEvents_list = []
lastIndexedDate_list = []

for x in df['data']:
    #print(x)
    id_list.append(x['id'])
    url_list.append(x['url']) 
    title_list.append(x['title']) 
    parkCode_list.append(x['parkCode'].upper()) 
    description_list.append(x['description'])
    category_list.append(x['category']) 
    relatedRoadEvents_list.append(x['relatedRoadEvents'])
    lastIndexedDate_list.append(x['lastIndexedDate'])


In [18]:
alerts_df = pd.DataFrame({
    'id': id_list, 
    'url': url_list, 
    'title': title_list,
    'park alpha': parkCode_list,
    'description': description_list,
    'category': category_list,
    'relatedRoadEvents':relatedRoadEvents_list,
    'LastIndexedDate':lastIndexedDate_list
})

In [19]:
alerts_df2 = alerts_df.merge(parks,left_on='park alpha', right_on="Alpha", how = "left")
alerts_df2.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit
0,3B7D50FE-A302-4CF6-8E89-2402458A5495,https://www.nps.gov/brca/planyourvisit/winter.htm,Winter Trail Closures,BRCA,The Wall St. side of Navajo Loop is expected t...,Park Closure,[],2024-04-02 14:02:04.0,Bryce Canyon,Bryce Canyon National Park,National Park,UT,IMR,BRCA,1.0
1,9840308A-02FB-4A2F-90A3-E5597EB3881B,https://www.nps.gov/brca/planyourvisit/conditi...,Main Road Status,BRCA,The main park road is fully open to Rainbow Po...,Information,[],2024-04-02 13:59:18.0,Bryce Canyon,Bryce Canyon National Park,National Park,UT,IMR,BRCA,1.0
2,80AA3666-78C7-4631-BCFD-DE5E57FB5238,,Water Line break at VC April 2nd 2024,KAHO,UPDATE: The visitor center will be open today ...,Information,[],2024-04-02 13:56:43.0,Kaloko-Honokohau,Kaloko-Honokohau National Historical Park,National Historical Park,HI,PWR,KAHO,1.0
3,EEE6CBBC-AC0D-41F9-A87C-F9C60C11321B,,Columns Drive entrance of Cochran Shoals Closed,CHAT,Columns Drive entrance of Cochran Shoals is cl...,Park Closure,[],2024-04-02 13:01:23.0,Chattahoochee River,Chattahoochee River National Recreation Area,National Recreation Area,GA,SER,CHAT,1.0
4,42D3786A-98AD-44B4-B212-D47038B708D8,,Active Bee Season,SAAN,Please use caution and be aware of your surrou...,Information,[],2024-04-02 12:21:39.0,San Antonio Missions,San Antonio Missions National Historical Park,National Historical Park,TX,IMR,SAAN,1.0


In [30]:
alerts_df2.shape

(699, 15)

In [34]:
road_words_list = ["road", "route", "interstate", "highway", "vehic"]

In [39]:
road_closed_flag_list = []


for x in range(len(alerts_df2)): 
    road_closed_flag = "No"
    
    for y in road_words_list:
        if y in alerts_df2["description"][x]:
            road_closed_flag = "Yes"
            break
            
    road_closed_flag_list.append(road_closed_flag)


In [41]:
alerts_df2["Road Closed Flag"] = road_closed_flag_list

In [43]:
alerts_df2.head()

,id,url,title,park alpha,description,category,relatedRoadEvents,LastIndexedDate,Park Name,Park Full Name,Type of Unit,State,Reg,Alpha,Unit,Road Closed Flag
0,3B7D50FE-A302-4CF6-8E89-2402458A5495,https://www.nps.gov/brca/planyourvisit/winter.htm,Winter Trail Closures,BRCA,The Wall St. side of Navajo Loop is expected t...,Park Closure,[],2024-04-02 14:02:04.0,Bryce Canyon,Bryce Canyon National Park,National Park,UT,IMR,BRCA,1.0,No
1,9840308A-02FB-4A2F-90A3-E5597EB3881B,https://www.nps.gov/brca/planyourvisit/conditi...,Main Road Status,BRCA,The main park road is fully open to Rainbow Po...,Information,[],2024-04-02 13:59:18.0,Bryce Canyon,Bryce Canyon National Park,National Park,UT,IMR,BRCA,1.0,Yes
2,80AA3666-78C7-4631-BCFD-DE5E57FB5238,,Water Line break at VC April 2nd 2024,KAHO,UPDATE: The visitor center will be open today ...,Information,[],2024-04-02 13:56:43.0,Kaloko-Honokohau,Kaloko-Honokohau National Historical Park,National Historical Park,HI,PWR,KAHO,1.0,No
3,EEE6CBBC-AC0D-41F9-A87C-F9C60C11321B,,Columns Drive entrance of Cochran Shoals Closed,CHAT,Columns Drive entrance of Cochran Shoals is cl...,Park Closure,[],2024-04-02 13:01:23.0,Chattahoochee River,Chattahoochee River National Recreation Area,National Recreation Area,GA,SER,CHAT,1.0,No
4,42D3786A-98AD-44B4-B212-D47038B708D8,,Active Bee Season,SAAN,Please use caution and be aware of your surrou...,Information,[],2024-04-02 12:21:39.0,San Antonio Missions,San Antonio Missions National Historical Park,National Historical Park,TX,IMR,SAAN,1.0,No


In [20]:
alerts_df2.to_csv("alerts_api.csv", index=False)

In [24]:
has_both

{'BLCA',
 'BUFF',
 'CRLA',
 'DEVA',
 'DINO',
 'GRSM',
 'GUMO',
 'MOJA',
 'ORPI',
 'OZAR',
 'PISC',
 'SHEN',
 'YOSE',
 'ZION'}

In [26]:
([1,2,3] * 5) 

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [28]:
alerts_df2.columns

Index(['id', 'url', 'title', 'park alpha', 'description', 'category',
       'relatedRoadEvents', 'LastIndexedDate', 'Park Name', 'Park Full Name',
       'Type of Unit', 'State', 'Reg', 'Alpha', 'Unit'],
      dtype='object')

In [29]:
merged_df.columns

Index(['date', 'park', 'alert name', 'description', 'event type', 'road name',
       'vehicle impact', 'event start date', 'alert_id', 'region', 'alpha',
       'State'],
      dtype='object')

In [27]:
has_alerts = set(alerts_df2['park alpha'])
has_tdx = set(merged_df['alpha'])
has_both = has_alerts.intersection(has_tdx)
only_alerts = has_alerts - has_tdx
only_tdx = has_tdx - has_alerts
status = pd.DataFrame(columns=['Park Alpha', 'Region', 'Status'])
status['Park Alpha'] = list(has_both) + list(only_alerts) + list(only_tdx)
status['Status'] = (['Using TDx feed and alerts'] * len(has_both)) + (['Only using alerts'] * len(only_alerts)) + (['Only using TDx feed'] * len(only_tdx))
status

,Park Alpha,Status
0,GUMO,Using TDx feed and alerts
1,YOSE,Using TDx feed and alerts
2,PISC,Using TDx feed and alerts
3,BUFF,Using TDx feed and alerts
4,ZION,Using TDx feed and alerts
5,CRLA,Using TDx feed and alerts
6,SHEN,Using TDx feed and alerts
7,MOJA,Using TDx feed and alerts
8,OZAR,Using TDx feed and alerts
9,DINO,Using TDx feed and alerts


In [22]:
status.to_csv("combined_status.csv", index=False)